## Data Exploration and Cleaning

In order to answer our research questions, we needed a variety of information such on topics such as population, GDP, emissions, temperatures, and natural disasters. In order to minimize missing data, we decided to pull from multiple data sources that were tailored around specific topics.

The four datasets we used:
 - **The World Bank DataBank** (URL: https://databank.worldbank.org/home.aspx)
    - *country, region, population, population growth, gdp, imports, exports*
 - **Our World in Data** (URL: https://ourworldindata.org/co2-and-other-greenhouse-gas-emissions)
    - *co2 emissions, methane emissions, nitrous oxide emissions, total ghg emissions*
 - **Kaggle** (URL: https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data)
    - *average temperatures for specific cities*
 - **EM-DAT International Disasters Database** (URL: https://www.emdat.be/)
    - *extreme weather events, locations, fatalities*

### General Statistics from The World Bank DataBank

There were a variety of cleaning tasks to perform on this dataset:
- Remove aggregate measures
- Combine countries.csv with factors.csv because the it has a column called 'regions' which will be helpful in analysis
- Rename columns
- Remove empty rows
- Rename all missing data from '..' to NA (will help with computations)

In [11]:
#import packages
import pandas as pd
import numpy as np

In [ ]:
#load data
factors = pd.read_csv("data/factors.csv")    #gdp, population, imports, exports
countries = pd.read_csv("data/countries.csv")    #country info

In [13]:
#remove aggregates from countries df
countries = countries[countries.Region != 'Aggregates']
#select columns of interest in countries df
countries_reduced = countries[["Country code", "Country name", "Region"]]
#rename country name in countries df to accomplish merge
countries_renamed = countries_reduced.rename(columns={"Country name":"Country Name"})
#delete country code and time code column froms factors df
factors_reduced = factors.drop(columns=["Country Code", "Time Code"])
#merge datasets on country name
data = pd.merge(countries_renamed, factors_reduced, on="Country Name")

In [14]:
#rename columns
data_renamed = data.rename(columns={'Country code': 'iso', 
                                    'Country Name': 'country', 
                                    'Region': 'region', 
                                    'Time': 'year', 
                                    'Time Code': 'year_code',
       'Population, total [SP.POP.TOTL]': 'pop_total',
       'Population growth (annual %) [SP.POP.GROW]': 'pop_growth',
       'GDP (constant 2010 US$) [NY.GDP.MKTP.KD]': 'gdp',
       'Imports of goods and services (constant 2010 US$) [NE.IMP.GNFS.KD]':'imports',
       'Exports of goods and services (current US$) [NE.EXP.GNFS.CD]':'exports',
       'Average precipitation in depth (mm per year) [AG.LND.PRCP.MM]':'avg_precip'})


In [46]:
#use numpy na to remove empty rows
data_renamed["iso"].replace('nan', np.nan, inplace=True)
data_renamed["iso"].replace('NaN', np.nan, inplace=True)
data_renamed.dropna(axis=0, subset=['iso'], inplace=True)
#replace '..' with numpy na
data_renamed.replace('..', np.nan, inplace=True)
#make copy with clean name
factors = data_renamed.copy()
#make sure year column is numeric
factors['year'] = factors['year'].astype(int)

### Emissions Data - Our World in Data

We didn't have to do much cleaning with this dataset, just had to subset to the years of interest and rename one column.

In [43]:
#load data
emissions_csv = pd.read_csv("data/emissions.csv")    #emissions of GHGs by country
#reduce emissions dataset to data of interest
##columns
emissions_subset = emissions_csv[['iso_code', 'country', 'year', 'co2', 'methane', 
                                  'nitrous_oxide', 'total_ghg']]
##years
emissions_reduced = emissions_subset[emissions_subset['year']>=2000]
##renamed iso_code to iso
emissions_reduced = emissions_reduced.rename(columns={'iso_code':'iso'})
##make copy with clean name
emissions = emissions_reduced.copy()
emissions

,iso,country,year,co2,methane,nitrous_oxide,total_ghg
51,AFG,Afghanistan,2000,0.758,10.59,3.62,15.05
52,AFG,Afghanistan,2001,0.798,9.36,3.22,13.47
53,AFG,Afghanistan,2002,1.052,11.21,3.72,16.10
54,AFG,Afghanistan,2003,1.186,11.56,3.92,16.78
55,AFG,Afghanistan,2004,0.889,11.47,3.82,16.35
...,...,...,...,...,...,...,...
23703,ZWE,Zimbabwe,2015,12.170,11.87,6.68,67.49
23704,ZWE,Zimbabwe,2016,10.815,11.92,6.55,65.98
23705,ZWE,Zimbabwe,2017,10.247,NaN,NaN,NaN
23706,ZWE,Zimbabwe,2018,11.341,NaN,NaN,NaN


### Extreme Weather Information - EM-DAT International Disaster Database

The main task with this dataset was selecting the columns of interest. There were 43 columns in the original dataset. Not all of them were formatted well or provided useful information so we had to parse through these and pick out only what was necessary.

In [22]:
#load data
disasters_csv = pd.read_csv("data/disasters.csv")    #database of occurences of natural disasters
#reduce disasters dataset to columns of interest
##columns
disasters_subset = disasters_csv[['Year','Country', 'ISO', 'Region', 'Continent', 
                                  'Disaster Group', 'Disaster Subgroup', 'Disaster Type', 
                                  'Disaster Subtype', 'Disaster Subsubtype','Total Deaths', 
                                  'Total Affected', 'Latitude', 'Longitude']]
##years of interest
disasters_reduced = disasters_subset[disasters_subset['Year']>=2000]
##make column names lowercase
disasters_reduced = disasters_reduced.rename(columns=str.lower)
##make copy with clean name
disasters = disasters_reduced.copy()

### Temperature Data - Kaggle

The temperature data was a bit more complicated in terms of wrangling. We weren't sure if we were going to graph anything using latitude/longitude, so we wanted to maintain these data points. The dataset which was organized by month didn't have latitude/longitude point available, so we took the dataset which was organized by city and summed the data by country.

In [23]:
#load data
temps_bycity_csv = pd.read_csv("data/temps_bycity.csv")    #land temperatures by city
#remove average temperature uncertainty column
temps_bycity_reduced = temps_bycity_csv[['dt','AverageTemperature','City', 'Country', 
                                         'Latitude', 'Longitude']]
#rename columns
temps_bycity_renamed = temps_bycity_reduced.rename(columns={'dt':'date',
                                                       'AverageTemperature': 'avg_temp_c',
                                                       'City': 'city',
                                                       'Country': 'country',
                                                       'Latitude': 'latitude',
                                                       'Longitude':'longitude'})
#create a temperature column in farenheit
temps_bycity_renamed['avg_temp_f'] = [((x*9/5) + 32) for x in temps_bycity_renamed['avg_temp_c']]

,date,avg_temp_c,city,country,latitude,longitude,avg_temp_f
0,1849-01-01,26.704,Abidjan,Côte D'Ivoire,5.63N,3.23W,80.0672
1,1849-02-01,27.434,Abidjan,Côte D'Ivoire,5.63N,3.23W,81.3812
2,1849-03-01,28.101,Abidjan,Côte D'Ivoire,5.63N,3.23W,82.5818
3,1849-04-01,26.140,Abidjan,Côte D'Ivoire,5.63N,3.23W,79.0520
4,1849-05-01,25.427,Abidjan,Côte D'Ivoire,5.63N,3.23W,77.7686
...,...,...,...,...,...,...,...
239172,2013-05-01,18.979,Xian,China,34.56N,108.97E,66.1622
239173,2013-06-01,23.522,Xian,China,34.56N,108.97E,74.3396
239174,2013-07-01,25.251,Xian,China,34.56N,108.97E,77.4518
239175,2013-08-01,24.528,Xian,China,34.56N,108.97E,76.1504


In [34]:
#create year and month columns
year_list = []
month_list = []
#for loop over date column
for date in temps_bycity_renamed['date']:
    split = date.split("-")
    y = pd.to_numeric(split[0])    #get year from string split
    m = pd.to_numeric(split[2])    #get month from string split
    year_list.append(y)    #append result to year list
    month_list.append(m)    #append result to month list
#turn lists in columns
temps_bycity_renamed['year'] = year_list
temps_bycity_renamed['month'] = month_list
#reorder columns
temps_bycity = temps_bycity_renamed[['year', 'month', 'date', 'city', 'country', 
                                     'avg_temp_c', 'avg_temp_f', 'latitude', 'longitude']]
#select years
temps_bycity = temps_bycity[temps_bycity['year']>=2000]

,year,month,date,city,country,avg_temp_c,avg_temp_f,latitude,longitude
49191,2000,1,2000-01-01,Changchun,China,-18.759,-1.7662,44.20N,125.22E
49192,2000,1,2000-02-01,Changchun,China,-13.105,8.4110,44.20N,125.22E
49193,2000,1,2000-03-01,Changchun,China,-1.089,30.0398,44.20N,125.22E
49194,2000,1,2000-04-01,Changchun,China,7.297,45.1346,44.20N,125.22E
49195,2000,1,2000-05-01,Changchun,China,16.970,62.5460,44.20N,125.22E
...,...,...,...,...,...,...,...,...,...
239019,2000,1,2000-08-01,Xian,China,22.517,72.5306,34.56N,108.97E
239020,2000,1,2000-09-01,Xian,China,17.823,64.0814,34.56N,108.97E
239021,2000,1,2000-10-01,Xian,China,11.116,52.0088,34.56N,108.97E
239022,2000,1,2000-11-01,Xian,China,4.095,39.3710,34.56N,108.97E


In [41]:
##create a dataframe of averaged temps by country
country_list = temps_bycity['country'].unique()
yr_list = temps_bycity['year'].unique()
temps_f_list = []
temps_c_list = []
country_col = []
year_col = []
#loop through countries
for i in range(len(country_list)):
    #loop through years
    for j in range(len(yr_list)):
        #subset to particular country and year
        c = country_list[i]
        y = yr_list[j]
        subset = temps_bycity[(temps_bycity['country']==c) & (temps_bycity['year']==y)]
        #average all temperature readings from that year
        avg_f = subset['avg_temp_f'].mean()
        avg_c = subset['avg_temp_c'].mean()
        #add averages to lists
        temps_f_list.append(avg_f)
        temps_c_list.append(avg_c)
        #add country names and years to lists
        country_col.append(c)
        year_col.append(y)
    
#create dataframe
temps_bycountry = pd.DataFrame({"country": country_col,
                                "year": year_col,
                               "avg_temp_c": temps_c_list,
                               "avg_temp_f": temps_f_list})
    

### Final Steps

The last step was to merge datasets together in a way that would allow each group member to address their research question and output these to CSV files.

In [50]:
#create dataframes that match each research questions
##Question 1
q1 = pd.merge(factors, emissions, on=['country','year', 'iso'])
q1_df = pd.merge(q1, temps_bycountry, on=['country', 'year'])
##Question 2
q2_df = pd.merge(emissions, temps_bycountry, on=['country','year'])
##Question 3
factors_subset = factors[['country', 'region', 'year', 'pop_total', 'pop_growth']]
q3_df = pd.merge(factors_subset, temps_bycountry, on=['country','year'])
##Question 4
factors_subset2 = factors[['country', 'region', 'year', 'avg_precip']]
q4 = pd.merge(factors_subset2, temps_bycountry, on=['country', 'year'])
q4_df = pd.merge(q4, emissions, on=['country', 'year'])

In [56]:
emissions.to_csv('data/emissions_clean.csv', index=False)
factors.to_csv('data/factors_clean.csv', index=False)
disasters.to_csv('data/disasters_clean.csv', index=False)
temps_bycity.to_csv('data/temps_bycity_clean.csv', index=False)
temps_bycountry.to_csv('data/temps_bycountry_clean.csv', index=False)
q1_df.to_csv('data/question1_df.csv', index=False)
q2_df.to_csv('data/question2_df.csv', index=False)
q3_df.to_csv('data/question3_df.csv', index=False)
q4_df.to_csv('data/question4_df.csv', index=False)